<a href="https://colab.research.google.com/github/JoyeBright/TxtStego-deep-learning/blob/master/SecretText_Module.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Secret Data**

In [1]:
text="this is a secret bit stream."
print("number of chars in secret bit:", len(text))

number of chars in secret bit: 28


# **ASCII Text to Binary**

In [2]:
import binascii

def text_to_bits(text, encoding='utf-8', errors='surrogatepass'):
    bits = bin(int(binascii.hexlify(text.encode(encoding, errors)), 16))[2:]
    return bits.zfill(8 * ((len(bits) + 7) // 8))

In [3]:
binary = (text_to_bits(text))

In [4]:
print("The binary bitsream:", binary)
print("Bistream length:", len(binary))
print("Bistream characters:",int(len(binary)/8))

The binary bitsream: 01110100011010000110100101110011001000000110100101110011001000000110000100100000011100110110010101100011011100100110010101110100001000000110001001101001011101000010000001110011011101000111001001100101011000010110110100101110
Bistream length: 224
Bistream characters: 28


# **Binary to Decimal**

In [5]:
def binToDec(n=binary):
   decimal = 0
   for d in binary:
     decimal = decimal*2 + int(d)
   return decimal

In [22]:
print("Decimal value of bistream:", binToDec())
print(len(str(binToDec())))

Decimal value of bistream: 12259178403199069570342758845700850558725039128334999982059829685550
68


# **Decimal to Binary (Test)**

In [7]:
def decToBin(n=binToDec()):
    if n==0: return ''
    else:
        return decToBin(n//2) + str(n%2)

In [8]:
print("Binary bitstream:", decToBin())
print("Bistream length:", len(decToBin()))

Binary bitstream: 1110100011010000110100101110011001000000110100101110011001000000110000100100000011100110110010101100011011100100110010101110100001000000110001001101001011101000010000001110011011101000111001001100101011000010110110100101110
Bistream length: 223


# **Binary to ASCII (Test)**

In [9]:
def text_from_bits(bits, encoding='utf-8', errors='surrogatepass'):
    n = int(bits, 2)
    return int2bytes(n).decode(encoding, errors)
def int2bytes(i):
    hex_string = '%x' % i
    n = len(hex_string)
    return binascii.unhexlify(hex_string.zfill(n + (n & 1)))

In [10]:
print(text_from_bits(decToBin()))

this is a secret bit stream.


# **Data Preparation**

*Take apart 4 digits and for them as a number*

In [19]:
print(binToDec())
digits = [int(d) for d in str(binToDec())]
#print(digits)
#print(len(digits))
num = []
for n in range(0, len(digits), 4):
  num.append(int(str(digits[n])+''+str(digits[n+1])+''+str(digits[n+2])+''+str(digits[n+3])))
signal = num
print(signal)
print(len(signal))

12259178403199069570342758845700850558725039128334999982059829685550
[1225, 9178, 4031, 9906, 9570, 3427, 5884, 5700, 8505, 5872, 5039, 1283, 3499, 9982, 598, 2968, 5550]
17


# **DWT**

## **1D Haar Wavelet Transform**

In [40]:
import pywt
print(pywt.Wavelet('haar'))
cA, cD = pywt.dwt(signal, 'haar', mode='zero')

Wavelet haar
  Family name:    Haar
  Short name:     haar
  Filters length: 2
  Orthogonal:     True
  Biorthogonal:   True
  Symmetry:       asymmetric
  DWT:            True
  CWT:            False


In [41]:
print(cA)
print(cD)

[ 7356.03184468  9854.9472094   9190.26683508  8191.12495326
 10166.07419312  4470.32907066  9532.50651718  2521.54278171
  3924.44263559]
[-5623.62023078 -4154.25233947  4343.75695683   130.10764774
  1861.81215486  2655.89307014 -4584.17326243 -1675.84307141
  3924.44263559]


## **Inverse Wavelet**

In [36]:
coe = pywt.idwt(cA, cD, 'haar')
coe = coe.astype(int)
print(pywt.idwt(cA, cD, 'haar'))
print(len(pywt.idwt(cA, cD, 'haar')))

[1225. 9178. 4031. 9906. 9570. 3427. 5884. 5700. 8505. 5872. 5039. 1283.
 3499. 9982.  598. 2968. 5550.    0.]
18


# **Cutomized 1D Haar Wavelet Transform**

In [20]:
def haarWavelet(signal):
  size = len(signal)
  s = []
  d = []
  index = []
  for n in range(0, size, 2):
    if(n==size-1): signal.append(signal[n]) #  padding to make a pair (rewrite the last number)
    s.append(int((signal[n]+signal[n+1])/(2)))
    if((signal[n]+signal[n+1])%2 !=0): index.append(n)
    d.append(int((signal[n]-signal[n+1])/(2)))
  return s, d, index

In [21]:
data=[9, 7, 3, 5, 21, 2, 2]
cS, cD, Index = haarWavelet(signal)
print(cS, cD)
print(Index)

[519, 696, 649, 579, 718, 315, 673, 177, 555] [-397, -293, 307, 9, 131, 187, -324, -118, 0]
[0, 2, 4, 8, 10, 12, 14]


# **Decimal To Binary (Again)**

In [44]:
import numpy as np
coefficient = np.concatenate((cS, cD), axis=None)
print(coefficient)
coefficient = coefficient.astype(str).tolist()
print(coefficient)
coefficient = ''.join(coefficient)
print(coefficient)
print(type(coefficient))


[ 519  696  649  579  718  315  673  177  555 -397 -293  307    9  131
  187 -324 -118    0]
['519', '696', '649', '579', '718', '315', '673', '177', '555', '-397', '-293', '307', '9', '131', '187', '-324', '-118', '0']
519696649579718315673177555-397-2933079131187-324-1180
<class 'str'>


In [40]:
bin = []
for n in range(0, len(coefficient)):
  bin.append(decToBin(coefficient[n]))
new_binary = "".join(bin)
print(new_binary)
print(len(new_binary))

TypeError: ignored

# **Inverse 1D Haar WT**

In [ ]:
def inverseHWT(S, D, I=[]):
  signal =[]
  size = len(S) 
  for n in range(0, size):
    signal.append(int((S[n]+D[n])))
    signal.append(int((S[n]-D[n])))
  index =np.zeros((size*2), dtype=int)
  for i in range(0, len(I)):
    index[I[i]+1] = 1
  signal +=index
  # decrease padding 
  if((S[-1]+D[-1])==S[-1] and D[-1]==0):
    signal = signal[:-1].astype(int)
    return signal
  else:  
    return signal.astype(int)

In [ ]:
print(inverseHWT(cS, cD))
print(len(inverseHWT(cS, cD)))
inverseWT = inverseHWT(cS, cD)

In [ ]:
# check whether they are equivalent or not
x = [107, 206, 739, 277, 866, 520, 959, 87, 52, 764, 226, 350, 359, 959, 547, 866, 920, 944, 416, 446, 884, 560, 707, 298, 65, 610, 374, 870, 839, 858, 248, 653, 938, 159, 556, 205, 523, 129, 360, 686, 662, 792, 669, 108, 879, 146, 888, 341, 666, 552, 130, 165, 340, 569, 58, 169, 593]
x = np.asarray(x) # next function needs same DS
print(type(x))
print(type(inverseWT))
print(np.array_equal(inverseWT, x))
print(inverseWT)
print(x)